In [43]:
from sshtunnel import SSHTunnelForwarder

# SSH config shortcut
SSH_CONFIG_SHORTCUT = 'myaws_SQL'

server = SSHTunnelForwarder(
    SSH_CONFIG_SHORTCUT,
    ssh_config_file='~/.ssh/config',
    remote_bind_address=('localhost', 5432)
)

server.start()
print(server.is_active, server.is_alive, server.local_bind_port)


True True 63583


In [44]:
from sqlalchemy import create_engine

# Postgres username, password, and database name
POSTGRES_IP_ADDRESS = 'localhost' ## This is localhost because SSH tunnel is active
POSTGRES_PORT = str(server.local_bind_port)
POSTGRES_USERNAME = 'ubuntu'     ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = '****' ## CHANGE THIS TO YOUR POSTGRES PASSWORD
POSTGRES_DBNAME = 'baseball'

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME, 
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_IP_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)

In [45]:
import pandas as pd

My tables(AllstarFull, Salaries, Schools, Franchises):

In [10]:
pd.read_sql_query('''SELECT * FROM allstarfull LIMIT 5;''', cnx)

,playerid,yearid,gamenum,gameid,teamid,lgid,gp,startingpos
0,gomezle01,1933,0,ALS193307060,NYA,AL,1,1
1,ferreri01,1933,0,ALS193307060,BOS,AL,1,2
2,gehrilo01,1933,0,ALS193307060,NYA,AL,1,3
3,gehrich01,1933,0,ALS193307060,DET,AL,1,4
4,dykesji01,1933,0,ALS193307060,CHA,AL,1,5


In [11]:
pd.read_sql_query('''SELECT * FROM salaries LIMIT 5;''', cnx)

,yearid,teamid,lgid,playerid,salary
0,1985,BAL,AL,murraed02,1472819.0
1,1985,BAL,AL,lynnfr01,1090000.0
2,1985,BAL,AL,ripkeca01,800000.0
3,1985,BAL,AL,lacyle01,725000.0
4,1985,BAL,AL,flanami01,641667.0


In [12]:
pd.read_sql_query('''SELECT * FROM schools LIMIT 5;''', cnx)

,schoolid,schoolname,schoolcity,schoolstate,schoolnick
0,abilchrist,Abilene Christian University,Abilene,TX,Wildcats
1,adelphi,Adelphi University,Garden City,NY,Panthers
2,adrianmi,Adrian College,Adrian,MI,Bulldogs
3,airforce,United States Air Force Academy,Colorado Springs,CO,Falcons
4,akron,University of Akron,Akron,OH,Zips


In [13]:
pd.read_sql_query('''SELECT * FROM franchises LIMIT 5;''', cnx)

,franchid,franchname,active,nassoc
0,ALT,Altoona Mountain City,N,None
1,ANA,Los Angeles Angels of Anaheim,Y,None
2,ARI,Arizona Diamondbacks,Y,None
3,ATH,Philadelphia Athletics,N,PNA
4,ATL,Atlanta Braves,Y,BNA


What was the total spent on salaries by each team, each year?

In [36]:
pd.read_sql_query('''SELECT teamid, yearid as year, sum(salary) as total_salary
                    FROM salaries
                    GROUP BY year, teamid
                    ORDER BY total_salary DESC
                    LIMIT 5
                    ''', cnx)

,teamid,year,total_salary
0,NYA,2013,231978886.0
1,LAN,2013,223362196.0
2,NYA,2005,208306817.0
3,NYA,2008,207896789.0
4,NYA,2010,206333389.0


What is the first and last year played for each player? Hint: Create a new table from 'Fielding.csv'.


In [52]:
pd.read_sql_query('''SELECT * FROM fielding LIMIT 5;''', cnx)

,playerid,yearid,stint,teamid,lgid,pos,g,gs,innouts,po,a,e,dp,pb,wp,sb,cs,zr
0,ansonca01,1871,1,RC1,NA,1B,1,None,None,7,0,0,0,None,None,None,None,None
1,biermch01,1871,1,FW1,NA,1B,1,None,None,9,0,2,0,None,None,None,None,None
2,carleji01,1871,1,CL1,NA,1B,29,None,None,295,4,34,10,None,None,None,None,None
3,connone01,1871,1,TRO,NA,1B,4,None,None,35,1,5,1,None,None,None,None,None
4,cravebi01,1871,1,TRO,NA,1B,2,None,None,18,1,1,1,None,None,None,None,None


In [51]:
pd.read_sql_query('''SELECT playerid as player, MIN(yearid) as first_year, MAX(yearid) as last_year 
                    FROM fielding
                    GROUP BY playerid
                    LIMIT 5
                    ''', cnx)

,player,first_year,last_year
0,gagnied01,1914,1915
1,ramospe01,1955,1970
2,putkolu01,2012,2013
3,boyerke01,1955,1969
4,dillibo01,1946,1951


Who has played the most all star games?

In [56]:
pd.read_sql_query('''SELECT playerid as player, COUNT(*) as Number_of_games
                     FROM allstarfull
                     GROUP BY playerid
                     ORDER BY Number_of_games DESC
                     LIMIT 1;''', cnx)

,player,number_of_games
0,aaronha01,25


Which players have the longest career? Assume that the debut and finalGame columns comprise the start and end, respectively, of a player's career. Hint: Create a new table from 'Master.csv'. Also note that strings can be converted to dates using the DATE function and can then be subtracted from each other yielding their difference in days.

In [57]:
pd.read_sql_query('''SELECT *
                     FROM Master
                     LIMIT 5''', cnx)

,playerid,birthyear,birthmonth,birthday,birthcountry,birthstate,birthcity,deathyear,deathmonth,deathday,...,namelast,namegiven,weight,height,bats,throws,debut,finalgame,retroid,bbrefid
0,aardsda01,1981,12,27,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,205,75,R,R,2004-04-06,2013-09-28,aardd001,aardsda01
1,aaronha01,1934,2,5,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180,72,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939,8,5,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190,75,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954,9,8,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190,75,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972,8,25,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184,73,L,L,2001-09-10,2006-04-13,abada001,abadan01


In [79]:
pd.read_sql_query('''SELECT playerid as player, (DATE(finalgame)  - DATE(debut)) as Days_in_career
                    FROM Master
                    WHERE (DATE(finalgame)  - DATE(debut)) IS NOT NULL
                    ORDER BY Days_in_career DESC
                     LIMIT 5''', cnx)

,player,days_in_career
0,altroni01,12862
1,orourji01,11836
2,minosmi01,11492
3,olearch01,11126
4,lathaar01,10678


What is the distribution of debut months? 
Hint: Look at the DATE and EXTRACT functions.


In [94]:
pd.read_sql_query('''SELECT COUNT(*), EXTRACT(MONTH FROM DATE(debut)) as Month
                     FROM Master
                     WHERE debut IS NOT NULL
                     GROUP BY month
                     ORDER BY count DESC
                     ''', cnx)

,count,month
0,5061,9.0
1,4711,4.0
2,2230,5.0
3,1978,7.0
4,1943,8.0
5,1893,6.0
6,308,10.0
7,41,3.0


What is the effect of table join order on mean salary for the players listed in the main (master) table? Hint: Perform two different queries, one that joins on playerID in the salary table and other that joins on the same column in the master table. Use Left joins since right joins are not currently supported with SQLalchemy.

In [103]:
pd.read_sql_query(r'''SELECT COUNT(*) as total_rows, AVG(salary) as average_salary
                     FROM salaries as s
                     LEFT JOIN Master as m
                     ON s.playerid = m.playerid
                    ''',cnx)

,total_rows,average_salary
0,23956,1.864357e+06


In [104]:
pd.read_sql_query(r'''SELECT COUNT(*) as total_rows, AVG(salary) as average_salary
                     FROM Master as m
                     LEFT JOIN salaries as s
                     ON m.playerid = s.playerid
                     ''',cnx)

,total_rows,average_salary
0,37642,1.864357e+06
